# Importation of libraries that will be used

In [1]:
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import sys
sys.path.append('../lib')
from functions import *

# Context

An A/B test was set into motion from `3/15/2017 to 6/20/2017`. This test is trying to prove wether or not a more intuitive and modern User Interface (UI), coupled with timely in-context prompts (cues, messages, hints, or instructions provided to users directly within the context of their current task or action), could make the online process smoother for clients. The critical question was: Would these changes encourage more clients to complete the process?


The goal is to see if the new design leads to a better user experience and higher process completion rates.



# Groups
**Control Group:** Clients interacted with the traditional online process.


**Test Group:** Clients experienced the new, spruced-up digital interface.

Both groups navigated through an identical process sequence: 
- an initial page 
- three subsequent steps
- a confirmation page signaling process completion.

# Datasets

- Client Profiles (df_demo): Demographics like age, gender, and account details of the clients.

- Experiment Roster (df_exp): A list revealing which clients were part of the grand experiment.

- Digital Footprints (df_web): A detailed trace of client interactions online, divided into two parts: pt_1 and pt_2 (they are merged into one called df_web).


# Data Reading and Cleaning

Read data from given raw files. 

**_WARNING:_** Do not move csv files that are on `..\..\data_files folder`


In [2]:
df_demo, df_exp, df_web = read_raw_data()

Drop na for experiment DF, as the client ids are not part of the experiment


In [3]:
df_exp = drop_na(df_exp)

Drop values that have more than 2 columns with null values. Fill  empty value via an imputation from the mean (only one occurence of this).


In [4]:
df_demo = drop_na(df_demo, 3)
df_demo = fill_na_mean(df_demo,'clnt_age')

# Merging & EDA

Create a new dataframe that merges the clients that are a part of thhe experiment with their demographic information, dropping the ones that have a null value in the Variation column (meaning that they are not a part pf the experiment)



In [5]:
df_online = pd.merge(df_demo,df_exp,how='left',on='client_id')
df_online.dropna(inplace=True)
df_online.head()

,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,Variation
0,836976,6.0,73.0,60.5,U,2.0,45105.30,6.0,9.0,Test
1,2304905,7.0,94.0,58.0,U,2.0,110860.30,6.0,9.0,Control
2,1439522,5.0,64.0,32.0,U,2.0,52467.79,6.0,9.0,Test
3,1562045,16.0,198.0,49.0,M,2.0,67454.65,3.0,6.0,Test
4,5126305,12.0,145.0,33.0,F,2.0,103671.75,0.0,3.0,Control


In [6]:
# check that there is no repetition of client_id
df_online['client_id'].nunique() == df_online.shape[0]

True

In [7]:
df_demo['clnt_age'].median()
# median of age is 47

47.0

In [8]:
df_online['clnt_age'].median()
# same median as well as average

48.0

In [9]:
df_online.describe() # numerical variables, client id info can be disregarded

,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,num_accts,bal,calls_6_mnth,logons_6_mnth
count,5.048800e+04,50488.000000,50488.000000,50488.000000,50488.000000,5.048800e+04,50488.000000,50488.000000
mean,5.006173e+06,12.031730,150.415485,47.319778,2.254575,1.495147e+05,3.093289,6.131873
std,2.877417e+06,6.860282,81.944830,15.518463,0.533671,3.020364e+05,2.187991,2.175423
min,5.550000e+02,2.000000,33.000000,17.000000,1.000000,2.378944e+04,0.000000,3.000000
25%,2.515700e+06,6.000000,82.000000,33.500000,2.000000,3.987841e+04,1.000000,4.000000
50%,5.025026e+06,11.000000,136.000000,48.000000,2.000000,6.573360e+04,3.000000,6.000000
75%,7.477918e+06,16.000000,192.000000,59.500000,2.000000,1.399565e+05,5.000000,8.000000
max,9.999832e+06,55.000000,669.000000,96.000000,7.000000,1.632004e+07,6.000000,9.000000


In [10]:
df_online.describe(include='object') # categorical variables

,gendr,Variation
count,50488,50488
unique,4,2
top,U,Test
freq,17280,26961


In [11]:
# check Test and Control counts
df_online['Variation'].value_counts()

Variation
Test       26961
Control    23527
Name: count, dtype: int64

In [12]:
df_web.head(10) # KPI - check who got to confirm

,client_id,visitor_id,visit_id,process_step,date_time
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04
5,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:17:15
6,9988021,580560515_7732621733,781255054_21935453173_531117,step_1,2017-04-17 15:17:01
7,9988021,580560515_7732621733,781255054_21935453173_531117,start,2017-04-17 15:16:22
8,8320017,39393514_33118319366,960651974_70596002104_312201,confirm,2017-04-05 13:10:05
9,8320017,39393514_33118319366,960651974_70596002104_312201,step_3,2017-04-05 13:09:43


In [13]:
df_web['process_step'].value_counts()

process_step
start      243945
step_1     163193
step_2     133062
step_3     112242
confirm    102963
Name: count, dtype: int64

In [14]:
df_web.info() # date_time is an object, need to change to datetime 

<class 'pandas.core.frame.DataFrame'>
Index: 755405 entries, 0 to 412263
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   client_id     755405 non-null  int64 
 1   visitor_id    755405 non-null  object
 2   visit_id      755405 non-null  object
 3   process_step  755405 non-null  object
 4   date_time     755405 non-null  object
dtypes: int64(1), object(4)
memory usage: 34.6+ MB


In [15]:
# Merge df_online with df_web
df_ab = pd.merge(df_online,df_web)
df_ab.head(12)

,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,Variation,visitor_id,visit_id,process_step,date_time
0,836976,6.0,73.0,60.5,U,2.0,45105.3,6.0,9.0,Test,427070339_1413275162,228976764_46825473280_96584,confirm,2017-04-02 11:51:13
1,836976,6.0,73.0,60.5,U,2.0,45105.3,6.0,9.0,Test,427070339_1413275162,228976764_46825473280_96584,confirm,2017-04-02 11:47:50
2,836976,6.0,73.0,60.5,U,2.0,45105.3,6.0,9.0,Test,427070339_1413275162,228976764_46825473280_96584,confirm,2017-04-02 11:46:45
3,836976,6.0,73.0,60.5,U,2.0,45105.3,6.0,9.0,Test,427070339_1413275162,228976764_46825473280_96584,step_3,2017-04-02 11:23:08
4,836976,6.0,73.0,60.5,U,2.0,45105.3,6.0,9.0,Test,427070339_1413275162,228976764_46825473280_96584,step_2,2017-04-02 11:22:24
5,836976,6.0,73.0,60.5,U,2.0,45105.3,6.0,9.0,Test,427070339_1413275162,228976764_46825473280_96584,step_1,2017-04-02 11:21:38
6,836976,6.0,73.0,60.5,U,2.0,45105.3,6.0,9.0,Test,427070339_1413275162,228976764_46825473280_96584,start,2017-04-02 11:21:28
7,836976,6.0,73.0,60.5,U,2.0,45105.3,6.0,9.0,Test,427070339_1413275162,104438405_2368283624_817211,start,2017-03-29 11:02:44
8,836976,6.0,73.0,60.5,U,2.0,45105.3,6.0,9.0,Test,427070339_1413275162,104438405_2368283624_817211,start,2017-03-29 11:01:40
9,836976,6.0,73.0,60.5,U,2.0,45105.3,6.0,9.0,Test,427070339_1413275162,104438405_2368283624_817211,start,2017-03-29 10:59:43


In [16]:
# Change date time col to dateime class
df_ab['date_time'] = pd.to_datetime(df_ab['date_time'])
df_ab.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 321207 entries, 0 to 321206
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   client_id         321207 non-null  int64         
 1   clnt_tenure_yr    321207 non-null  float64       
 2   clnt_tenure_mnth  321207 non-null  float64       
 3   clnt_age          321207 non-null  float64       
 4   gendr             321207 non-null  object        
 5   num_accts         321207 non-null  float64       
 6   bal               321207 non-null  float64       
 7   calls_6_mnth      321207 non-null  float64       
 8   logons_6_mnth     321207 non-null  float64       
 9   Variation         321207 non-null  object        
 10  visitor_id        321207 non-null  object        
 11  visit_id          321207 non-null  object        
 12  process_step      321207 non-null  object        
 13  date_time         321207 non-null  datetime64[ns]
dtypes: d

In [17]:
# split DF by test and control groups
df_test, df_control = split_df(df_ab)

# Success Metric (KPIs)
### Completion Rate 
The proportion of users who reach the final ‘confirm’ step.

In [19]:
completion_rate_ctl, confirm_step_count_ctl, total_users_ctl = completion_rate(df_control)
print(f"Completion Rate Control Group: {completion_rate_ctl:.2%}, Test Total: {total_users_ctl}, Confirm Count: {confirm_step_count_ctl} ")

Completion Rate Control Group: 65.58%, Test Total: 23527, Confirm Count: 15429 


In [21]:
completion_rate_tst, confirm_step_count_tst, total_users_tst = completion_rate(df_test)
print(f"Completion Rate Test Group : {completion_rate_tst:.2%}, Test Total: {total_users_tst}, Confirm Count: {confirm_step_count_tst} ")

Completion Rate Test Group : 69.29%, Test Total: 26961, Confirm Count: 18682 


In [25]:
zscore, p_value = sm.stats.proportions_ztest(confirm_step_count_tst, total_users_tst, known_proportion)
zscore, p_value

(68.67457427861147, 0.0)

###  Time Spent on Each Step
The average duration users spend on each step. Need to check with Raiana

In [41]:
# transform colunm to datetime 
# order by client and date
# and then shift with diferrence for new column
#df_ab = df_ab.sort_values(['client_id','date_time'], key=lambda x: pd.to_datetime(x))
df_ab = df_ab.sort_values(['client_id','date_time'])
df_ab['first_time']= df_ab.groupby('client_id')['date_time'].shift(1)
df_ab['previous_step']= df_ab.groupby('client_id')['process_step'].shift(1)
df_ab.dtypes

client_id                     int64
clnt_tenure_yr              float64
clnt_tenure_mnth            float64
clnt_age                    float64
gendr                        object
num_accts                   float64
bal                         float64
calls_6_mnth                float64
logons_6_mnth               float64
Variation                    object
visitor_id                   object
visit_id                     object
process_step                 object
date_time            datetime64[ns]
first_time           datetime64[ns]
diff                timedelta64[ns]
previous_step                object
dtype: object

In [42]:
# take diff
df_ab['diff'] = df_ab['date_time'] - df_ab['first_time']

In [43]:
df_test, df_control = split_df(df_ab)

In [44]:
df_ab.groupby('process_step')['diff'].mean()

process_step
confirm   1 days 00:40:48.170193941
start     3 days 15:03:36.993693593
step_1    0 days 00:25:56.563424249
step_2    0 days 00:01:18.518814981
step_3    0 days 00:11:02.429579202
Name: diff, dtype: timedelta64[ns]

In [45]:
df_test.groupby('process_step')['diff'].mean()

process_step
confirm   1 days 05:00:41.146959592
start     3 days 02:55:37.780723214
step_1    0 days 00:22:20.321410097
step_2    0 days 00:01:51.238731757
step_3    0 days 00:12:19.069989559
Name: diff, dtype: timedelta64[ns]

In [46]:
df_control.groupby('process_step')['diff'].mean()

process_step
confirm   0 days 18:21:25.156749327
start     4 days 06:00:00.843642966
step_1    0 days 00:30:39.337504650
step_2    0 days 00:00:39.347493332
step_3    0 days 00:09:35.350907253
Name: diff, dtype: timedelta64[ns]

In [49]:
df_control_time_each_step = filter_steps(df_control)
df_test_time_each_step = filter_steps(df_test) 

In [50]:
df_control_time_each_step

,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,Variation,visitor_id,visit_id,process_step,date_time,first_time,diff,previous_step
302426,1028,12.0,145.0,36.0,M,3.0,103520.22,1.0,4.0,Control,42237450_62128060588,557292053_87239438319_391157,step_1,2017-04-08 18:52:17,2017-04-08 18:51:28,0 days 00:00:49,start
302424,1028,12.0,145.0,36.0,M,3.0,103520.22,1.0,4.0,Control,42237450_62128060588,557292053_87239438319_391157,step_2,2017-04-08 18:53:29,2017-04-08 18:53:20,0 days 00:00:09,step_1
302423,1028,12.0,145.0,36.0,M,3.0,103520.22,1.0,4.0,Control,42237450_62128060588,557292053_87239438319_391157,step_3,2017-04-08 18:58:04,2017-04-08 18:53:29,0 days 00:04:35,step_2
302420,1028,12.0,145.0,36.0,M,3.0,103520.22,1.0,4.0,Control,42237450_62128060588,557292053_87239438319_391157,step_2,2017-04-08 19:00:17,2017-04-08 19:00:15,0 days 00:00:02,step_1
284895,1186,8.0,99.0,22.0,U,2.0,31662.52,0.0,3.0,Control,446844663_31615102958,795373564_99931517312_810896,step_1,2017-04-08 18:05:13,2017-04-08 18:05:02,0 days 00:00:11,start
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142977,9998346,50.0,602.0,61.5,F,2.0,149881.38,6.0,9.0,Control,292425655_16607136645,189177304_69869411700_783154,step_2,2017-03-29 15:26:47,2017-03-29 15:26:02,0 days 00:00:45,step_1
142974,9998346,50.0,602.0,61.5,F,2.0,149881.38,6.0,9.0,Control,292425655_16607136645,189177304_69869411700_783154,step_3,2017-03-29 15:28:00,2017-03-29 15:27:49,0 days 00:00:11,step_2
142972,9998346,50.0,602.0,61.5,F,2.0,149881.38,6.0,9.0,Control,292425655_16607136645,189177304_69869411700_783154,step_2,2017-03-29 15:30:11,2017-03-29 15:29:55,0 days 00:00:16,step_1
142971,9998346,50.0,602.0,61.5,F,2.0,149881.38,6.0,9.0,Control,292425655_16607136645,189177304_69869411700_783154,step_3,2017-03-29 15:30:25,2017-03-29 15:30:11,0 days 00:00:14,step_2


In [51]:
df_test_time_each_step

,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,Variation,visitor_id,visit_id,process_step,date_time,first_time,diff,previous_step
255407,555,3.0,46.0,29.5,U,2.0,25454.66,2.0,6.0,Test,402506806_56087378777,637149525_38041617439_716659,step_1,2017-04-15 12:58:03,2017-04-15 12:57:56,0 days 00:00:07,start
255406,555,3.0,46.0,29.5,U,2.0,25454.66,2.0,6.0,Test,402506806_56087378777,637149525_38041617439_716659,step_2,2017-04-15 12:58:35,2017-04-15 12:58:03,0 days 00:00:32,step_1
255405,555,3.0,46.0,29.5,U,2.0,25454.66,2.0,6.0,Test,402506806_56087378777,637149525_38041617439_716659,step_3,2017-04-15 13:00:14,2017-04-15 12:58:35,0 days 00:01:39,step_2
255404,555,3.0,46.0,29.5,U,2.0,25454.66,2.0,6.0,Test,402506806_56087378777,637149525_38041617439_716659,confirm,2017-04-15 13:00:34,2017-04-15 13:00:14,0 days 00:00:20,step_3
9748,647,12.0,151.0,57.5,M,2.0,30525.80,0.0,4.0,Test,66758770_53988066587,40369564_40101682850_311847,step_1,2017-04-12 15:41:35,2017-04-12 15:41:28,0 days 00:00:07,start
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310108,9999729,10.0,124.0,31.0,F,3.0,107059.74,6.0,9.0,Test,834634258_21862004160,870243567_56915814033_814203,step_1,2017-05-08 16:08:30,2017-05-08 16:08:25,0 days 00:00:05,start
310107,9999729,10.0,124.0,31.0,F,3.0,107059.74,6.0,9.0,Test,834634258_21862004160,870243567_56915814033_814203,step_2,2017-05-08 16:08:40,2017-05-08 16:08:30,0 days 00:00:10,step_1
310106,9999729,10.0,124.0,31.0,F,3.0,107059.74,6.0,9.0,Test,834634258_21862004160,870243567_56915814033_814203,step_3,2017-05-08 16:09:19,2017-05-08 16:08:40,0 days 00:00:39,step_2
310105,9999729,10.0,124.0,31.0,F,3.0,107059.74,6.0,9.0,Test,834634258_21862004160,870243567_56915814033_814203,confirm,2017-05-08 16:09:40,2017-05-08 16:09:19,0 days 00:00:21,step_3


# Hypothesis Testing

## Completion Rate
Null Hypthesis: There is no difference between the completion rate between the Test Group and the control Group

Alt Hyphoesis: There is a difference between the completion rate of the Test Group and the control Group



In [32]:
# using chi-squaer ince its categorical variable
data = [[confirm_step_count_tst, total_users_tst  * (1-completion_rate_tst)],
        [confirm_step_count_ctl, total_users_ctl * (1-completion_rate_ctl)]]
# Perform Chi-Square test
chi2, p_value, _, _ = stats.chi2_contingency(data)
# Output the results
print(f"Chi-Square Statistic: {chi2}")
print(f"P-Value: {p_value}")
# Check the significance level (e.g., 0.05)
alpha = 0.05
# Make a decision based on the p-value
if p_value < alpha:
    print("Reject the null hypothesis: There is a significant difference in completion rates between the test and control groups.")
else:
    print("Fail to reject the null hypothesis: There is no significant difference in completion rates between the test and control groups.")

Chi-Square Statistic: 78.8519479876901
P-Value: 6.694339983392046e-19
Reject the null hypothesis: There is a significant difference in completion rates between the test and control groups.


In [33]:
# Other way
crosstable = pd.DataFrame({ "Control": [total_users_ctl-confirm_step_count_ctl, confirm_step_count_ctl],
              "Test": [total_users_tst-confirm_step_count_tst, confirm_step_count_tst]}, index= ['Not Completed','Completed'])
# and use crosstable instead of data 
crosstable

,Control,Test
Not Completed,8098,8279
Completed,15429,18682


## Completion Rate with a Cost-Effectiveness Threshold
Null Hypthesis: There is no difference between the completion rate between the Test Group and the control Group

Alt Hyphoesis: There is a difference between the completion rate of the Test Group and the control Group

In [47]:
stat, p_value = sm.stats.proportions_ztest([confirm_step_count_ctl, confirm_step_count_tst], [total_users_ctl, total_users_tst])
stat, p_value

(-8.889390699380462, 6.144491429497383e-19)